<a href="https://colab.research.google.com/github/Akuvasney/ProjetosPLN/blob/main/Atividade4_grupo9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2023.Q3]**
Prof. Alexandre Donizeti Alves

### **ATIVIDADE PRÁTICA 04 [Uso da API da OpenAI com técnicas de PLN]**


A **ATIVIDADE PRÁTICA 04** deve ser feita utilizando o **Google Colab** com uma conta sua vinculada ao Gmail. O link do seu notebook, armazenado no Google Drive, além do link de um repositório no GitHub e os principais resultados da atividade, devem ser enviados usando o seguinte formulário:

> https://forms.gle/GzwCq3R7ExtE9g9a8


**IMPORTANTE**: A submissão deve ser feita até o dia **26/11 (domingo)** APENAS POR UM INTEGRANTE DA EQUIPE, até às 23h59. Por favor, lembre-se de dar permissão de ACESSO IRRESTRITO para o professor da disciplina de PLN.

### **EQUIPE**

---

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01:**

`Amanda Melati Kuvasney 11201811735`

**Integrante 02:**

`Beatriz Domingos de Oliveira 11202130893`

**Integrante 03:**

`Gabriel Gomes de Oliveira Costa  11201921471`

### **LIVRO**
---

`Processamento de Linguagem Natural - Conceitos, Técnicas e Aplicações em Português.`

>

Disponível gratuitamente em:
  
  > https://brasileiraspln.com/livro-pln/1a-edicao/.


**POR FAVOR, PREENCHER OS CAPITULOS SELECIONADOS PARA A SUA EQUIPE:**

`Primeiro capítulo: 09`

`Segundo capítulo: 17`



### **DESCRIÇÃO**
---

Implementar um `notebook` no `Google Colab` que faça uso da **API da OpenAI** aplicando, no mínimo, 3 técnicas de PLN. As técnicas devem ser aplicadas nos 2 (DOIS) capítulos do livro **Processamento de Linguagem Natural - Conceitos, Técnicas e Aplicações em Português**.

>

**RESTRIÇÃO**: É obrigatório usar o *endpoint* "*`Chat Completions`*".

>

As seguintes técnicas de PLN podem ser usadas:

*   Correção Gramatical
*   Classificação de Textos
*   Análise de Sentimentos
*   Detecção de Emoções
*   Extração de Palavras-chave
*   Tradução de Textos
*   Sumarização de Textos
*   **Similaridade de Textos**
*   **Reconhecimento de Entidades Nomeadas**
*   **Sistemas de Perguntas e Respostas**

>

Os capítulos devem ser os mesmos selecionados na **ATIVIDADE PRÁTICA 02**. Para consultar os capítulos, considere a seguinte planilha:

>

> https://docs.google.com/spreadsheets/d/1ZutzQ3v1OJgsgzCvCwxXlRIQ3ChXNlHNvB63JQvYsbo/edit?usp=sharing

>
>

**IMPORTANTE:** É obrigatório usar o e-mail da UFABC. Não é permitido alterar os capítulos já selecionados.



### **CRITÉRIOS DE AVALIAÇÃO**
---


Serão considerados como critérios de avaliação as técnicas usadas e a criatividade envolvida na aplicação das mesmas.




### **IMPLEMENTAÇÃO**
---

In [ ]:
# Instalando as dependências
!pip install openai==0.28.1
!pip install requests beautifulsoup4

In [ ]:
#@title Versão da API da OpenAI

import openai

print(openai.__version__)

0.28.1


In [ ]:
import openai
import os

#capturando apikey pela variavel de sistema
os.environ["OPENAI_API_KEY"] = openai.api_key

#testando a conexão com openai
resposta = openai.Completion.create(
   model = "text-davinci-003",
   prompt = "O Brasil é um país"
)

print(resposta.choices[0].text)

 na América do Sul. Ele é o 5º país mais


#Introdução

**Desafios da implantação:**

1.   **Tamanho do texto:** Pelo texto ser muito grande, a API não suportou fazer diversas análises no texto inteiro. Sendo assim, quebramos ele em partes para fazermos as análises de maneira faseada, conforme será demonstrado nas técnicas abaixo. Além disso, em algumas funções foi utilizado o parâmetro "max_tokens=algum numero" definindo o número de tokens que ele retornará de acordo com a necessidade.

2.   **Limite de tempo:** em alguns casos, a API estoura o tempo máximo de processamento, então utilizamos "time.sleep(20)" em cada iteração para não ocorrer esse problema.



In [ ]:
# @title Separando o conteúdo dos capítulos em partes de até 4000 caracteres
import requests
from bs4 import BeautifulSoup
import re


#define function to scrap information about the page
def scrap_text_information(url) :
   response = requests.get(url);
   soup = BeautifulSoup(response.content, 'html.parser')
   paragraphs = soup.find_all('p')
   scraped_text = '\n'.join(paragraph.get_text() for paragraph in paragraphs)     #correcao bia

   #scraped_text = soup.get_text()                                                #codigo origem
   return scraped_text

def get_paragraphs(text) :
  paragraph_pattern = r'([A-Z].*?\n)'
  # Use re.findall to find all paragraph-like patterns
  paragraphs = re.findall(paragraph_pattern, text)
  return paragraphs


# a function that returns True if letter is vowel
def valid_sub_chapter(sub_chapter):
    return len(sub_chapter) < 400;

def extract_chapters(url):

    # Define a regular expression pattern for matching chapter headings
    chapter_pattern = re.compile(r'^\d+(\.\d+)+\s+[^\n]+')

    response = requests.get(url);
    soup = BeautifulSoup(response.content, 'html.parser')

    lines = soup.get_text().split('\n')

    # Extract content between chapter headings
    chapters = []
    current_chapter = None

    for line in lines:
        if chapter_pattern.match(line):
            # If a new chapter heading is found, start a new chapter
            if current_chapter is not None:
                chapters.append(current_chapter.strip())
            current_chapter = f"{line}\n"
        elif current_chapter is not None:
            # Add lines to the current chapter content
            current_chapter += f"{line}\n"

    # Add the last chapter
    if current_chapter is not None:
        chapters.append(current_chapter.strip())

    chapters = list(filter(lambda x: len(x) >= 400, chapters))

    return chapters

In [ ]:
import re
import time

def split_text_equally(text):
    # Find the index of the middle newline character
    middle_index = len(text) // 2
    newline_index = text.rfind('\n', 0, middle_index)

    # If there is no newline character in the first half, search in the second half
    if newline_index == -1:
        newline_index = text.find('\n', middle_index)

    # If there is still no newline character, just split the text in the middle
    if newline_index == -1:
        newline_index = middle_index

    # Split the text at the newline_index
    part1 = text[:newline_index]
    part2 = text[newline_index:]

    return part1, part2

def concatenate_into_parts(paragraphs, num_parts=4):
    total_paragraphs = len(paragraphs)
    paragraphs_per_part = total_paragraphs // num_parts

    # Calculate the number of paragraphs in the last part
    remaining_paragraphs = total_paragraphs % num_parts

    # Initialize variables to keep track of the starting and ending index for each part
    start_index = 0
    end_index = 0

    concatenated_parts = []  # List to store concatenated parts

    for part in range(1, num_parts + 1):
        # Calculate the ending index for the current part
        end_index += paragraphs_per_part + (1 if part <= remaining_paragraphs else 0)

        # Concatenate paragraphs for the current part
        part_paragraphs = paragraphs[start_index:end_index]
        concatenated_part = '\n'.join(part_paragraphs)

        # Append the concatenated part to the list
        concatenated_parts.append(concatenated_part)

        # Update the starting index for the next part
        start_index = end_index

    return concatenated_parts

In [ ]:
# @title
# Initialize the constants
URL_CHAPTER_9 = 'https://brasileiraspln.com/livro-pln/1a-edicao/parte5/cap9/cap9.html'
URL_CHAPTER_17 = 'https://brasileiraspln.com/livro-pln/1a-edicao/parte8/cap17/cap17.html'

In [ ]:
text_chapther_9 = scrap_text_information(URL_CHAPTER_9)
text_chapther_17 = scrap_text_information(URL_CHAPTER_17)

In [ ]:
sub_chapters_from_chapter_9 = extract_chapters(URL_CHAPTER_9)
sub_chapters_from_chapter_17 = extract_chapters(URL_CHAPTER_17)

# Erros de ortográfia

Separamos os textos dos capítulos 9 e 17 em 8 partes para realizarmos a análise, pois estavámos atingindo o limite de tokens estabelecido.
Além disso, foi necessário colocar uma função `time.sleep(20)` após cada interação com a API, tendo em vista que estavámos recebendo um erro de `RateLimitError: Rate limit reached`.
O modelo não detectou erros de ortográfia em nenhuma parte do texto, mas percebemos que em algumas chamadas ocorre alguns falsos negativos, é detectado erroneamente um erro de ortográfia e a correção é exatamente o mesmo texto.

In [ ]:
# check if there is a date in the format MM/DD/YYYY or YYYY/MM/DD as the article is in PT-BR so the dates must be DD/MM/YYYY
import re
import time

def get_spelling_errors(text) :
   paragraphs = get_paragraphs(text)
   # separate the text into 8 parts
   concanate_paragraphs = concatenate_into_parts(paragraphs, 8)
   count = 0
   for paragraph in concanate_paragraphs :
      res = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Retorne apenas os erros de ortográfia com a sua respectiva correção, se não existir erro na frase retornar 'NÃO EXISTE ERROS', no seguinte texto :\n'{paragraph}",
        max_tokens=100,  # Ajuste conforme necessário
    )
      print(f'PARTE {count + 1} : \n')
      print(res['choices'][0]['text'].strip())
      time.sleep(20)
      count = count + 1


In [ ]:
get_spelling_errors(text_chapther_9)

PARTE 1 : 

NÃO EXISTEM ERROS
PARTE 2 : 

NÃO EXISTE ERROS
PARTE 3 : 

NÃO EXISTE ERRO
PARTE 4 : 

NÃO EXISTE ERROS
PARTE 5 : 

NÃO EXISTEM ERROS
PARTE 6 : 

NÃO EXISTE ERROS.
PARTE 7 : 

NÃO EXISTE ERROS.
PARTE 8 : 

NÃO EXISTE ERROS


In [ ]:
get_spelling_errors(text_chapther_17)

PARTE 1 : 

NÃO EXISTE ERROS
PARTE 2 : 

NÃO EXISTE ERROS
PARTE 3 : 

NÃO EXISTE ERRO
PARTE 4 : 

NÃO EXISTE ERROS
PARTE 5 : 

NÃO EXISTE ERROS.
PARTE 6 : 

NÃO EXISTE ERRO
PARTE 7 : 

NÃO EXISTE ERRO
PARTE 8 : 

NÃO EXISTE ERRO.


# Palavras chaves




As palavras chaves foram processadas para cada parte do capítulo considerando a limitação de caracteres existente na versão gratuita da API Openai

**Separando o capítulo em 8 partes e buscando as palavras chaves em cada parte**

In [ ]:
def get_key_words(text) :
   paragraphs = get_paragraphs(text)
   # separate the text into 8 parts
   concanate_paragraphs = concatenate_into_parts(paragraphs, 8)
   count = 0
   for paragraph in concanate_paragraphs :
      res = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Retorne as palavras chaves do seguinte texto :\n'{paragraph}",
        max_tokens=100,  # Ajuste conforme necessário
    )
      print(f'PARTE {count + 1} : \n')
      print(res['choices'][0]['text'].strip())
      time.sleep(20)
      count = count + 1

In [ ]:
get_key_words(text_chapther_17)

PARTE 1 : 

Palavras Chaves: Extração de Informação, Reconhecimento de Entidades Nomeadas, Extração de Relação, Extração de Eventos, Parsers Sintáticos, Gabarito, Regras, Message Understanding Conference, Métricas de Avaliação.
PARTE 2 : 

Palavras-chave: Extração de Informação, Entidade, Relação, Processamento de Linguagem Natural, Relacionamento Ontológico, Aprendizagem de Máquina.
PARTE 3 : 

Palavras-chave: Extração de Informação, Extração de Informação Aberta, Reconhecimento de Entidades Nomeadas, Extração de Relações, Entidades, Relações, Escopo, EI Tradicional, Aprendizado de Máquina, Arquiteturas Neurais.
PARTE 4 : 

Palavras-Chave: REN, regras lexico-sintáticas, almanaques, classificadores, classificação sequencial, redes neurais profundas, BART, RoBERTa, T5, BERT, GPT-3, HAREM, MALINCHE, NEURA, R3M, RELP-CRF, Word Embeddings, Flair Embeddings, Flair
PARTE 5 : 

Palavras Chaves: Corpus, Rede Neural BiLSTM-CRF, Flair Embeddings, Padrões de Hearst, Aprendizado de Máquina, REN, E

In [ ]:
get_key_words(text_chapther_9)

PARTE 1 : 

Palavras-chaves: Processamento de Linguagem Natural (PLN), Métodos Simbólicos, Analisadores Semânticos, Representação Lógica, Bases de Conhecimento, Sistemas de Inteligência Artificial (IA).
PARTE 2 : 

Palavras-chave: WordNet, PLN, semântica de linguagem natural, redes semânticas, bases de conhecimento léxico-semânticas, bases de conhecimento de senso comum, recursos léxico-semânticos, Frames, ConceptNet, George A. Miller, Christiane Fellbaum.
PARTE 3 : 

• “assalto” (00783063-n) → “roubo” (01089816-n) 
• “terminar” (02610845-v) → “acabar” (00038128-v)

Palavras-chave: WordNet, Princeton, várias, línguas, português, recursos léxico-semânticos, Wordnet.BR, Onto.PT, PULO, OpenWordNet-PT, PWN, OWN-PT, hiperônimo, Desambiguação do Sentido de Palavras, WSD.
PARTE 4 : 

Palavras chaves: OWN-PT, FrameNet, Commiting_crime, Semantic Role Labeling, WordNet, PropBank, VerbNet, Natural Language Inference, SemLink, FrameNet Brasil, Cometer_crime.
PARTE 5 : 

Palavras chaves : FrameNet,

**Buscando as palavras chaves por subcapítulo**

In [ ]:
def split_text_equally(text):
    # Find the index of the middle newline character
    middle_index = len(text) // 2
    newline_index = text.rfind('\n', 0, middle_index)

    # If there is no newline character in the first half, search in the second half
    if newline_index == -1:
        newline_index = text.find('\n', middle_index)

    # If there is still no newline character, just split the text in the middle
    if newline_index == -1:
        newline_index = middle_index

    # Split the text at the newline_index
    part1 = text[:newline_index]
    part2 = text[newline_index:]

    return part1, part2

In [ ]:
def get_key_words_by_subchapter(subchapters) :
   count = 0
   for subchapter in subchapters :
      if (len(subchapter) > 4800) :
          text1, text2 = split_text_equally(subchapter);
          res = openai.Completion.create(
              model="text-davinci-003",
              prompt=f"Retorne as palavras chaves do seguinte texto :\n'{text1}",
              max_tokens=100,
          )
          print(f'Subcapítulo {count + 1} [pt1] : \n')
          print(res['choices'][0]['text'].strip())
          time.sleep(20)
          res = openai.Completion.create(
              model="text-davinci-003",
              prompt=f"Retorne as palavras chaves do seguinte texto :\n'{text1}",
              max_tokens=100,
          )
          print(f'Subcapítulo {count + 1} [pt2] : \n')
          print(res['choices'][0]['text'].strip())
      else :
        res = openai.Completion.create(
              model="text-davinci-003",
              prompt=f"Retorne as palavras chaves do seguinte texto :\n'{subchapter}",
              max_tokens=100,
          )
        print(f'Subcapítulo {count + 1} : \n')
        print(res['choices'][0]['text'].strip())
      time.sleep(20)
      count = count + 1

In [ ]:
get_key_words_by_subchapter(sub_chapters_from_chapter_9)

Subcapítulo 1 [pt1] : 

Palavras-chave:Processamento de Linguagem Natural (PLN), Métodos Simbólicos, Analisadores Semânticos, Representação Semântica, Motores de Inferência, Bases de Conhecimento, Engenharia de Ontologia, Formalismo de Troca de Dados.
Subcapítulo 1 [pt2] : 

Palavras-chaves: Processamento de Linguagem Natural (PLN), Semântica, Técnicas Simbólicas, Analisador Semântico, Representação Semântica, Motor de Inferência, Sintaxe, Ontologias, Reconhecimento de Entidades Nomeadas.
Subcapítulo 2 [pt1] : 

Palavras chaves: Inteligência Artificial, redes semânticas, bases de conhecimento computáveis, frame, PLN, recursos léxico-semânticos, wordnet, PWN, bases de conhecimento de senso comum.
Subcapítulo 2 [pt2] : 

Palavras-Chave: Inteligência Artificial, Redes Semânticas, Representações baseadas em Frames, PLN, Léxico-Semânticos, WordNet, PWN, Senso Comum.
Subcapítulo 3 : 

Palavras-chave: WordNet, Princeton WordNet, synsets, PWN, APIs, Google Scholar, GitHub, relações semânticas,

#Similiaridade de textos


In [ ]:
def similarity_parties(part1, part2) :
  paragraphs1 = get_paragraphs(part1)
  paragraphs2 = get_paragraphs(part2)

  concanate_paragraphs1 = concatenate_into_parts(paragraphs1, 8);
  concanate_paragraphs2 = concatenate_into_parts(paragraphs2, 8);

  cont=0

  for paragraph1 in concanate_paragraphs1:
    cont = cont+1
    for paragraph2 in concanate_paragraphs2:
      print("parte do cap 9")
      print(c)
      print(paragraph1)
      print("parte do cap 17\n")
      print(paragraph2)


similarity_parties(text_chapther_9, text_chapther_17)


parte do cap 9

Eloize Rossi Marques Seno 

Valéria de Paiva 

Vládia Pinheiro 

PDF

Métodos Simbólicos em Processamento de Linguagem Natural (PLN) envolvem a utilização de regras e representações formais explícitas para processar e entender textos em linguagem natural. Esses métodos especializam-se na manipulação de símbolos e dados estruturados, como gramáticas, ontologias e bases de conhecimento. Especificamente para o entendimento de textos em linguagem natural usando técnicas simbólicas, existem analisadores semânticos (ou parsers semânticos) e bases de conhecimento semântico, que visam fornecer uma representação semântica dos textos. A partir desta representação, motores de inferência são capazes de realizar raciocínio para que aplicações possam, por exemplo, extrair informações, sumarizar textos, e responder perguntas com base nos textos.

A Figura 9.1 apresenta uma arquitetura tradicional para sistemas de entendimento de textos em linguagem natural (Natural Language Understand

In [ ]:
import openai

# Função para comparar dois parágrafos usando a API OpenAI
def comparar_paragrafos(paragrafo1, paragrafo2):
    prompt = f"Comparar o parágrafo 1:\n{paragrafo1}\nCom o parágrafo 2:\n{paragrafo2}\n"
    resposta = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        max_tokens=100,  # Ajuste conforme necessário
    )
    return resposta.choices[0].text.strip()

# Parágrafos de exemplo (substitua pelos parágrafos reais)
paragrafo1 = "Este é o primeiro parágrafo de exemplo."
paragrafo2 = "Este é o segundo parágrafo de exemplo."

# Comparar os parágrafos
resultado_comparacao = comparar_paragrafos(paragrafo1, paragrafo2)

# Imprimir o resultado
print(f"Resultado da comparação:\n{resultado_comparacao}")


Resultado da comparação:
Os parágrafos 1 e 2 são iguais pois ambos são parágrafos de exemplo. A única diferença está na numeração e na primeira palavra, pois o primeiro parágrafo começa com o "Este" enquanto o segundo parágrafo inicia com o "Esse".


#Tradução de textos

Tentando contornar as limitações da API gratuita, em que ficaria inviável traduzir o texto todo, foi feita a tradução apenas das palavras chave, demonstradas abaixo:

In [ ]:
def translated_key_words(text) :
   paragraphs = get_paragraphs(text)
   # separate the text into 8 parts
   concanate_paragraphs = concatenate_into_parts(paragraphs, 8)
   count = 0
   for paragraph in concanate_paragraphs :
      res = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Retorne as palavras chaves do seguinte texto traduzidas para inglês:\n'{paragraph}",
        max_tokens=100,  # Ajuste conforme necessário
    )
      print(f'PARTE {count + 1} : \n')
      print(res['choices'][0]['text'].strip())
      time.sleep(20)
      count = count + 1

In [ ]:
translated_key_words(text_chapther_9)

PARTE 1 : 

Palavras-chave: Natural Language Understanding, Natural Language Processing, Symbolic Methods, Semantic Parsers, Knowledge Bases, Description Logics, First-Order Logic, Prolog, Intensional Logics, Abstract Meaning Representation, DELPH-IN, WordNet, FrameNet, ConceptNet, WikiData, YAGO, BabelNet.

Natural Language Understanding, Natural Language Processing, Symbolic Methods, Semantic Parsers, Knowledge Bases, Description Log
PARTE 2 : 

Palavras-chave: Natural Language Processing (PLN), léxico-semânticos, bases de conhecimento de senso comum, WordNet, desambiguação de sentido, Princeton WordNet, FrameNet, ConceptNet.

Keywords: Natural Language Processing (PLN), lexical-semantics, common-sense knowledge bases, WordNet, sense disambiguation, Princeton WordNet, Frame
PARTE 3 : 

WordNet of Princeton, various wordnets proposed for various languages, among them Portuguese, as described in the following section.

Several lexico-semantic resources have been created for Portuguese 

In [ ]:
translated_key_words(text_chapther_17)

PARTE 1 : 

Palavras-chave: Extraction of Information, Named Entity Recognition, Relationship Extraction, Event Extraction, Message Understanding Conference, 1980s, 1990s.
PARTE 2 : 

Palavras-chave: MUC-3, 1991, Processamento de Textos Jornalísticos, DARPA, Relação REN, Nomes Próprios, Aprendizado de Máquina, Aprendizado Semi-Supervisionado, DIPRE, Snowball, Wikipédia, Freebase, Rede Neural, Deep Learning, Grandes Modelos de Linguagem,
PARTE 3 : 

Palavras-chave: Entity Extraction, Relation Extraction, Open Information Extraction, Entity Recognition, Named Entity Recognition, Relation Recognition, Brazil Geography, Knowledge Representation, Knowledge-rich Methods, Data-rich Methods, Machine Learning, Neural Networks.
PARTE 4 : 

Keywords: Initial approaches, lexico-syntactic rules, gazeeers, names, word lists, linguistic feature identification, data-driven methods, classification-based methods, neural networks based methods, specialized domains.
PARTE 5 : 

Palavras-chave: Portuguese 

#Sumarização de textos

Para a sumarização, foi requisitado a API que explicasse o conteúdo do texto resumidamente.
Comparativamente, foi pedido que resumisse o texto.
A API deu um contexto melhor do que se tratava cada pedaço ao pedir que explicasse o texto.

In [ ]:
def sumarize_text(text) :
   paragraphs = get_paragraphs(text)
   # separate the text into 8 parts
   concanate_paragraphs = concatenate_into_parts(paragraphs, 8)
   count = 0
   for paragraph in concanate_paragraphs :
      res = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Explique o conteúdo do seguinte texto resumidamente:\n'{paragraph}",
        max_tokens=1000,  # Ajuste conforme necessário
    )
      print(f'TEXTO RESUMIDO - PARTE {count + 1} : \n')
      print(res['choices'][0]['text'].strip())
      time.sleep(20)
      count = count + 1

In [ ]:
sumarize_text(text_chapther_9)

TEXTO RESUMIDO - PARTE 1 : 

O texto descreve o uso da lógica e das bases de conhecimento no Processamento de Linguagem Natural (PLN). Esses métodos se especializam na manipulação de símbolos e dados estruturados, como gramáticas, ontologias e bases de conhecimento, para processar e entender palavras em linguagem natural. O texto discute como esses métodos podem ser usados ​​para várias tarefas, desde extrair informações e resumir textos até responder a perguntas. O texto também explica os diferentes tipos de lógicas e bases de conhecimento que podem ser usados, como WordNet e FrameNet.
TEXTO RESUMIDO - PARTE 2 : 

Esse texto discorre sobre as bases de conhecimento relevantes para o Processamento de Linguagem Natural (PLN). Ele apresenta detalhes a respeito da WordNet, uma base léxico-semântica desenvolvida por George A. Miller, Christiane Fellbaum e colaboradores, para o inglês. Esta base de conhecimento organiza os itens lexicais (palavras ou expressões) em synsets. A Figura 9.2 most

In [ ]:
sumarize_text(text_chapther_17)

TEXTO RESUMIDO - PARTE 1 : 

Resumidamente, esse texto faz referência à Extração de Informação (EI) com o objetivo de se obter informação estruturada de dados não-estruturados. Esse termo é frequentemente dividido em diversas tarefas de interesse, com foco no tipo de informação a ser extraída do texto. Elas incluem Reconhecimento de Entidades Nomeadas (REN), Extração de Relações (ER) e Extração de Eventos (EE). É explicado um pouco da história da EI e as conferências MUC, onde foram definidos aspectos cruciais para a área.
TEXTO RESUMIDO - PARTE 2 : 

Este texto mostra os desenvolvimentos que aconteceram na área de Extração de Informação e Reconhecimento de Entidades Nomeadas desde a década de 1990, descrevendo as abordagens mais antigas baseadas em padrões e semi-supervisão, até as mais contemporâneas baseadas em redes neurais. É mostrada ainda a importância da definição das noções de Entidades e Relações para que a tarefa possa ser realizada.

O conteúdo deste texto trata sobre a evo

In [ ]:
def resume_text(text) :
   paragraphs = get_paragraphs(text)
   # separate the text into 8 parts
   concanate_paragraphs = concatenate_into_parts(paragraphs, 8)
   count = 0
   for paragraph in concanate_paragraphs :
      res = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Resuma o conteúdo do seguinte texto:\n'{paragraph}",
        max_tokens=1000,  # Ajuste conforme necessário
    )
      print(f'TEXTO RESUMIDO - PARTE {count + 1} : \n')
      print(res['choices'][0]['text'].strip())
      time.sleep(20)
      count = count + 1

In [ ]:
resume_text(text_chapther_9)

TEXTO RESUMIDO - PARTE 1 : 

Ao lidar com textos em PLN, os métodos simbólicos envolvem a utilização de regras e representações formais para processar linguagem natural. Existem analisadores semânticos e bases de conhecimento semântico, que visam fornecer uma representação semântica dos textos. O processamento sintático lida com detecção de língua, separação de sentenças, tokenização, análise morfológica e sintática. A análise semântica se divide entre reconhecimento de entidades e identificação de expressões, e usa sistemas lógicos que dependem da forma lógica do texto. Além disso, os conceitos relacionados contribuem para a inferência, para além da forma da sentença. As bases de conhecimento mais usadas são WordNet, OpenWordNet-PT, FrameNet e FrameNet Brasil, além das ontologias WikiData, YAGO, BabelNet e as bases de conhecimento voltadas ao senso comum, como ConceptNet, OMCS-BR e InferenceNet-BR. 

Métodos simbólicos em PLN envolvem a utilização de regras e representações formais pa

In [ ]:
resume_text(text_chapther_17)

TEXTO RESUMIDO - PARTE 1 : 

Resumidamente, a Extração de Informação (EI) consiste em processar informação não-estruturada e obter informação estruturada. Os primeiros trabalhos remontam à década de 1970 com a aplicação de gramáticas formais e parsers sintáticos. Dentre as tarefas de EI mais abordadas na literatura estão o Reconhecimento de Entidades Nomeadas (REN), Extração de Relações (ER) e Extração de Eventos (EE). As conferências MUC realizadas na década de 1990, promovidas pela DARPA, foram cruciais para definir aspectos centrais da área.
TEXTO RESUMIDO - PARTE 2 : 

Resumo: Na área de Processamento de Linguagem Natural foram apresentados avanços tecnológicos no desenvolvimento de aplicações de Extração de Informação e Reconhecimento de Entidades a partir dos anos 1990, com o uso de dados e análise estatística, semi-supervisão e aprendizado de máquina, além de deep learning. Estes métodos foram fundamentais para a extração de relações entre entidades e extração de informações abe